In [4]:
import os
import pickle as pkl
import sys

import numpy as np
import random as rn
import argparse
from itertools import count
from collections import defaultdict

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn import preprocessing

from tqdm import tqdm

## Part 4

In [10]:
# Load Label
devel_label_file = "../lab3_part1/corpus/labels/dev_labels.csv"
labels_devel = pd.read_csv(devel_label_file, sep=',')

# prediction files -> EDIT THESE PATHS TO YOUR PREDICTIONS
dev_1_pred = 'is11_dev_result_svm.csv'
test_1_pred = 'is11_test_result_svm.csv'

dev_2_pred = 'is11_dev_result_nn.csv'
test_2_pred = 'is11_test_result_nn.csv'

dev_3_pred = 'egemaps_dev_result_svm.csv'
test_3_pred = 'egemaps_test_result_svm.csv'

# Load the predictions by speaker obtained with get_predictions_by_speaker.py
preds_devel_1 = pd.read_csv(dev_1_pred)
preds_devel_1.columns=['file_id', 'predictions_1']
preds_test_1 = pd.read_csv(test_1_pred)
preds_test_1.columns=['file_id', 'predictions_1']

preds_devel_2 = pd.read_csv(dev_2_pred)
preds_devel_2.columns=['file_id', 'predictions_2']
preds_test_2 = pd.read_csv(test_2_pred)
preds_test_2.columns=['file_id', 'predictions_2']

preds_devel_3 = pd.read_csv(dev_3_pred)
preds_devel_3.columns=['file_id', 'predictions_3']
preds_test_3 = pd.read_csv(test_3_pred)
preds_test_3.columns=['file_id', 'predictions_3']


# Merge all predictions and labels (if available) as columns of the same dataframe:
devel = pd.merge(pd.merge(pd.merge(labels_devel, preds_devel_1, on='file_id'), preds_devel_2, on='file_id'), preds_devel_3, on="file_id")
test = pd.merge(pd.merge(preds_test_1, preds_test_2, on='file_id'),  preds_test_3, on="file_id")

n_predictors = 3

# TODO: compute the majority vote
devel['mv'] = devel[devel.columns[-n_predictors:]].sum(axis=1)/n_predictors > 0.5
devel["mv"] = devel["mv"].astype(int)
test['mv'] = test[test.columns[-n_predictors:]].sum(axis=1)/n_predictors > 0.5
test["mv"] = test["mv"].astype(int)

devel.to_csv("dev_result_fusion.csv", index=False)
test.to_csv("test.result.fusion.csv", index=False)

# Print out the results for each model and for the final combination
print("Results for the Development Dataset")

print("Results for prediction 1")
f1 = precision_recall_fscore_support(devel.label.values, devel.predictions_1.values, labels=[0,1], average='macro')
print(f1)

print("Results for tprediction 2")
f1 = precision_recall_fscore_support(devel.label.values, devel.predictions_2.values, labels=[0,1], average='macro')
print(f1)

print("Results for tprediction 3")
f1 = precision_recall_fscore_support(devel.label.values, devel.predictions_3.values, labels=[0,1], average='macro')
print(f1)

print("Results for majority vote")
f1 = precision_recall_fscore_support(devel.label.values, devel.mv.values, labels=[0,1], average='macro')
print(f1)

Results for the Development Dataset
Results for prediction 1
(0.5025163834053261, 0.5027017395157096, 0.4974962945849505, None)
Results for tprediction 2
(0.5206951528626322, 0.5160033518907361, 0.5074969553907376, None)
Results for tprediction 3
(0.5024612693653173, 0.5026090327676216, 0.5000593202596134, None)
Results for majority vote
(0.4953513956046768, 0.4956247230837395, 0.4944605367399485, None)
